Converting the price and the units into € / 100 gramms

In [38]:
import pandas as pd
import numpy as np

feb = pd.read_csv('/Users/david/Documents/GitHub/Final_project_DGU_LDU_COFFEE/2_data_cleaning&analysis/1_cleaning/cleaned_coffee_df.csv')

price = feb['est_price']
price = pd.DataFrame(price)
price['est_price'] = feb['est_price']
price['blind_assessment'] = feb['blind_assessment'] 

We move in a new column everything after the '/' in a new column

In [37]:
split_values = price['est_price'].str.split('/')
# Create a new column 'unit' with the second part of the split values
price['after_slash'] = split_values.str[1]

We move in a new column everything before the '/' in a new column

In [3]:
price['before_slash'] = price['est_price'].apply(lambda x: x.split('/')[0])

We move in a new column everything before the '/' in a new column

In [4]:
price['before_DOL'] = price['est_price'].apply(lambda x: x.split('$')[0])

In [5]:
#We remove the currency symbol
split_values = price['before_slash'].str.split('$')

# Create a new column 'price_new' with the second part of the split values
price['rem_D'] = split_values.str[1]

In [6]:
replace_dict = {'NT': '', '.': ','}
price['rem_D'] = price['rem_D'].replace(replace_dict)

In [7]:
price.loc[1893, 'rem_D']

'NT 700'

In [8]:
#One string is still reminding :
price.loc[1893, 'rem_D'] = price.loc[1893, 'rem_D'].replace('NT', '', 1893)

In [9]:
price['rem_D'].dtypes

dtype('O')

In [10]:
price['rem_D'] = price['rem_D'].str.strip()

In [11]:
# We identify the empty value which provide a ""
empty_rows = price[price['rem_D'] == '']
empty_rows

,est_price,blind_assessment,after_slash,before_slash,before_DOL,rem_D
170,$NT$520/200g,"Resonantly floral, fruit-driven. Narcissus, po...",200g,$NT$520,,
171,$NT$1000/200g,"Sweet-tart, gently herbaceous. Dried apricot, ...",200g,$NT$1000,,


In [12]:
price.loc[171, 'rem_D'] = 1000
price.loc[170, 'rem_D'] = 520

In [13]:
price.dropna(subset=['rem_D'], inplace=True)

In [14]:
price['rem_D'] = price['rem_D'].str.replace(',', '')

In [15]:
price.loc[217, 'rem_D'] = price.loc[217, 'rem_D'].replace(',', '', 217)
price.loc[219, 'rem_D'] = price.loc[219, 'rem_D'].replace(',', '', 219)

In [16]:
price.loc[217, 'rem_D']

'1500'

In [17]:
price.loc[219, 'rem_D']

'1500'

We will move the rest of the String from 'rem_D' to 'before_DOL'

In [18]:
price['rem_D'] = pd.to_numeric(price['rem_D'], errors='coerce')

In [19]:
price['rem_D'] = price['rem_D'].astype(float)

In [20]:
# For CAD Canadien Dollar
for index, row in price.iterrows():
    if isinstance(row['rem_D'], float):
        # Handle NaN or other float values as needed
        continue

    if 'CAD' in row['rem_D']:
        row['before_DOL'] += 'CAD '
        row['rem_D'] = row['rem_D'].replace('CAD', '').strip()
# Same for Laotian Kip Currency

for index, row in price.iterrows():
    if isinstance(row['rem_D'], float):
        # Handle NaN or other float values as needed
        continue

    if 'LAK' in row['rem_D']:
        row['before_DOL'] += 'LAK '
        row['rem_D'] = row['rem_D'].replace('LAK', '').strip()
        
# Same for New Taiwan dollar Currency

for index, row in price.iterrows():
    if isinstance(row['rem_D'], float):
        # Handle NaN or other float values as needed
        continue

    if 'NTD' in row['rem_D']:
        row['before_DOL'] += 'NTD '
        row['rem_D'] = row['rem_D'].replace('NTD', '').strip()

In [21]:
#To simplify the database we drop the rest of the record which lack of informations
missing_count = price['rem_D'].isna().sum()
missing_count

7

In [22]:
price.dropna(subset=['rem_D'], inplace=True)

**Currency convertion into EURO**

In [23]:
# First, let´s check the uniques currency
price['before_DOL'].unique()

array(['', 'NT ', 'NT', 'CAD ', 'HKD ', 'HK ', 'US ', 'USD ', 'IDR ',
       'NTD ', 'AED ', 'AUD ', 'KRW ', 'THB ', 'RMB ', 'CNY ', 'TWD ',
       'KRW', 'NTD', 'Price: '], dtype=object)

In [24]:
#We create a column named 'Euro'
price['Euro'] = pd.Series('', index=price.index)

In [25]:
# converting the US Dollar in price['Euro']
# US Dollar to EURO (0,95): 'US ', 'USD ', 'Price'
#price.loc[price['before_DOL'].str.contains('US\s*|USD\s*|Price\s*', regex=True), 'Euro'] = price['rem_D'] * 0.95
price['Euro'] = price.apply(lambda x: x['rem_D']*0.95 if pd.isna(x['before_DOL']) or x['before_DOL'] == '' else x['Euro'], axis=1)

In [26]:
# New Taiwan dollar to Euro (0,031 Euro) :'NT ', 'NT', 'NTD ', 'NTD', 'TWD'
price.loc[price['before_DOL'].str.contains('NT\s*|NTD\s*|TWD\s*', regex=True), 'Euro'] = price['rem_D'] * 0.031

# CAD dollar to Euro (0.69): 'CAD '
price.loc[price['before_DOL'].str.contains('CAD ', regex=True), 'Euro'] = price['rem_D'] * 0.69

# Hong Kong Dollar to Euro (0,12): 'HKD ','HK '
price.loc[price['before_DOL'].str.contains('HKD\s*|HK\s*', regex=True), 'Euro'] = price['rem_D'] * 0.12

# Indonesian Rupiah to Euro (0,000061):'IDR ',
price.loc[price['before_DOL'].str.contains('IDR\s*', regex=True), 'Euro'] = price['rem_D'] * 0.000061

# United Arab Emirates Dirham to Euro (0,26):'AED ',
price.loc[price['before_DOL'].str.contains('AED\s*', regex=True), 'Euro'] = price['rem_D'] * 0.26

# Australian Dollar to Euro (0,63):'AUD ',
price.loc[price['before_DOL'].str.contains('AUD\s*', regex=True), 'Euro'] = price['rem_D'] * 0.63

# South Korean won to Euro (0,00072):'KRW ',
price.loc[price['before_DOL'].str.contains('KRW\s*', regex=True), 'Euro'] = price['rem_D'] * 0.00072

# Thai Baht to Euro (0,027):'THB ',
price.loc[price['before_DOL'].str.contains('THB\s*', regex=True), 'Euro'] = price['rem_D'] * 0.027

# Chinese Yuan to Euro (0,14):'RMB ', 'CNY'
price.loc[price['before_DOL'].str.contains('RMB\s*|CNY\s*', regex=True), 'Euro'] = price['rem_D'] * 0.14

**Weight units convertion into 100 gramms**

In [28]:
#We create a column named '100_gr_EUR'
price['100_gr_EUR'] = pd.Series('', index=price.index)

In [30]:
#We look for missing value and drop them
missings_count = price['after_slash'].isna().sum()
missings_count

4

In [36]:
price.dropna(subset=['after_slash'], inplace=True)

In [29]:
price['100_gr_EUR'].info

<bound method Series.info of 0        
1        
2        
3        
4        
       ..
4944     
4945     
4946     
4947     
4948     
Name: 100_gr_EUR, Length: 4874, dtype: object>

In [42]:
price.to_csv('Price_weight_cleaned.csv')

In [32]:
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*1.67 if any(substring in row['after_slash'] for substring in ['60 grams']) else row['100_gr_EUR'], axis=1)

TypeError: argument of type 'float' is not iterable

In [33]:
# , 8 ounces,' 8 ounces',' 8 oz.', 8 oz., 227 grams , 227g, ' 227 grams'
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.44 if any(substring in row['after_slash'] for substring in ['8 ounces','8-ounce','8 ounces','227 grams','227g','227 grams',' 8 oz.', '8 oz.']) else row['100_gr_EUR'], axis=1)

# 4 ounces , 4 ounces, 120 grams, 115 grams #WORK
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.83 if any(substring in row['after_slash'] for substring in ['4 ounces','4 ounces','120 grams','115 grams']) else row['100_gr_EUR'], axis=1)

# 10 ounces, 285 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.35 if any(substring in row['after_slash'] for substring in ['10 ounces','285 grams']) else row['100_gr_EUR'], axis=1)

# 6 ounces, 
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.59 if any(substring in row['after_slash'] for substring in ['6 ounces']) else row['100_gr_EUR'], axis=1)

# 200 grams, 200g, 200 g., 7 ounces, 7 oz.,
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.50 if any(substring in row['after_slash'] for substring in ['200 grams','200 g.','200g','7 ounces','7 oz.','7 oz.']) else row['100_gr_EUR'], axis=1)

# 60 grams, 
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*1.67 if any(substring in row['after_slash'] for substring in ['60 grams']) else row['100_gr_EUR'], axis=1)

# 16 ounces, ' 16 ounces', ' 16 ounces.', 16ounces,' 16 ounces.',16 oz, 454 grams, 453 grams, 460 g,
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.22 if any(substring in row['after_slash'] for substring in ['16 ounces','16 ounces','453 grams','16ounces','16 ounces.','16 oz','454 grams.','460 grams.']) else row['100_gr_EUR'], axis=1)

# 230 grams, 230g
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.44 if any(substring in row['after_slash'] for substring in ['230 grams','230g']) else row['100_gr_EUR'], axis=1)

# 150 grams, 150-gram, 5.5 ounces
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.67 if any(substring in row['after_slash'] for substring in ['150 grams','150-gram','5.5 ounces']) else row['100_gr_EUR'], axis=1)

# 225 grams, 225 gram,' 225 grams',
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.44 if any(substring in row['after_slash'] for substring in ['225 gram','225 grams']) else row['100_gr_EUR'], axis=1)

# 14 ounces, 400 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.25 if any(substring in row['after_slash'] for substring in ['14 ounces','400 grams']) else row['100_gr_EUR'], axis=1)

# 240 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.42 if any(substring in row['after_slash'] for substring in ['240 grams']) else row['100_gr_EUR'], axis=1)

# 250 g., 250 gram, ' 250 g.',250 g., 8.8 oz, 8.8 ounces
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.40 if any(substring in row['after_slash'] for substring in ['250 g','250 gram','250 g','8.8','8.8 ounces']) else row['100_gr_EUR'], axis=1)

# 100 grams, 3.5 ounces, 105 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*1 if any(substring in row['after_slash'] for substring in ['100 grams','3.5 ounces','105 grams']) else row['100_gr_EUR'], axis=1)

# 50 gram, 50 g. 
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*2.00 if any(substring in row['after_slash'] for substring in ['50 gram','50 g. ']) else row['100_gr_EUR'], axis=1)

# 20 grams, 21 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*5.00 if any(substring in row['after_slash'] for substring in ['20 grams','21 grams']) else row['100_gr_EUR'], axis=1)

# 175 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.57 if any(substring in row['after_slash'] for substring in ['175 grams']) else row['100_gr_EUR'], axis=1)

# 10.5 ounces, 300 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.33 if any(substring in row['after_slash'] for substring in ['10.5 ounces','300 grams']) else row['100_gr_EUR'], axis=1)

# 5 ounces
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.71 if any(substring in row['after_slash'] for substring in ['5 ounces']) else row['100_gr_EUR'], axis=1)

# 220 grams,
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.45 if any(substring in row['after_slash'] for substring in ['220 grams']) else row['100_gr_EUR'], axis=1)

# 5.5 ounces
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.64 if any(substring in row['after_slash'] for substring in ['5.5 ounces']) else row['100_gr_EUR'], axis=1)

# 500 grams, 500 grams, 500 g. 
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.20 if any(substring in row['after_slash'] for substring in ['500 grams','500 grams','500 g.']) else row['100_gr_EUR'], axis=1)

# 12.64 ounces, 
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.28 if any(substring in row['after_slash'] for substring in ['12.64 ounces']) else row['100_gr_EUR'], axis=1)

# 11 ounces
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.32 if any(substring in row['after_slash'] for substring in ['11 ounces']) else row['100_gr_EUR'], axis=1)

# 20 ounces, 
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.18 if any(substring in row['after_slash'] for substring in ['20 ounces']) else row['100_gr_EUR'], axis=1)

# 32 ounces,
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.11 if any(substring in row['after_slash'] for substring in ['32 ounces']) else row['100_gr_EUR'], axis=1)

# 22 ounces,
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.16 if any(substring in row['after_slash'] for substring in ['22 ounces']) else row['100_gr_EUR'], axis=1)

# 2 ounces,
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*1.79 if row['after_slash'] == '2 ounces' else row['100_gr_EUR'], axis=1)

# 30 ounces, 2 pounds, 
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.12 if any(substring in row['after_slash'] for substring in ['30 ounces','2 pounds']) else row['100_gr_EUR'], axis=1)

# 70 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*1.43 if any(substring in row['after_slash'] for substring in [' 70 grams']) else row['100_gr_EUR'], axis=1)

# 24 ounces
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.15 if any(substring in row['after_slash'] for substring in ['24 ounces']) else row['100_gr_EUR'], axis=1)

# 36 ounces
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.10 if any(substring in row['after_slash'] for substring in ['36 ounces']) else row['100_gr_EUR'], axis=1)

# 11.3 ounces, 320 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.31 if any(substring in row['after_slash'] for substring in ['11.3 ounces','320 grams']) else row['100_gr_EUR'], axis=1)

# 28 ounces
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.13 if any(substring in row['after_slash'] for substring in ['28 ounces']) else row['100_gr_EUR'], axis=1)

# 10.3 ounces'
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.34 if any(substring in row['after_slash'] for substring in ['10.3 ounces']) else row['100_gr_EUR'], axis=1)

# 13 ounces
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.27 if any(substring in row['after_slash'] for substring in ['13 ounces']) else row['100_gr_EUR'], axis=1)

# 165 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.61 if any(substring in row['after_slash'] for substring in ['165 grams']) else row['100_gr_EUR'], axis=1)

# 6 pounds
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.59 if any(substring in row['after_slash'] for substring in ['6 pounds']) else row['100_gr_EUR'], axis=1)

# 1 kilogram, 1 kg
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.10 if any(substring in row['after_slash'] for substring in ['1 kg','320 grams']) else row['100_gr_EUR'], axis=1)

# 0.94 oz, 0.93 oz
#price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*3.85 if any(substring in row['after_slash'] for substring in ['0.94 oz.','0.93 oz']) else row['100_gr_EUR'], axis=1)
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*3.85 if row['after_slash'] in ['0.94 oz.','0.93 oz'] else row['100_gr_EUR'], axis=1)

# 340 grams
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.29 if any(substring in row['after_slash'] for substring in ['340 grams']) else row['100_gr_EUR'], axis=1)

# 32 oz.
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.11 if any(substring in row['after_slash'] for substring in ['32 oz.']) else row['100_gr_EUR'], axis=1)

# 12 ounces : '12 ounce', 12 ounces, 12-ounce bottle, 12 oz, 12 onces, 340, 12 ouncues
price['100_gr_EUR'] = price.apply(lambda row: row['Euro']*0.29 if any(substring in row['after_slash'] for substring in ['12 ounces','12 oz','12 oz','12-ounce','12 ounce','12 onces','12 ouncues']) else row['100_gr_EUR'], axis=1)

TypeError: can't multiply sequence by non-int of type 'float'

In [30]:
#price['after_slash'] = str(price['after_slash'])
#price['after_slash'] = price['after_slash'].astype(str)
#price['Euro'] = price['Euro'].astype(str)
price['100_gr_EUR'] = price['100_gr_EUR'].astype(str)

In [45]:
print(price['after_slash'].apply(type).value_counts())

<class 'str'>    4870
Name: after_slash, dtype: int64


In [46]:
## First, let´s check the uniques units
price['after_slash'].unique()

array(['12 ounces', '8 ounces', '4 ounces', '227 grams', '10 ounces',
       '6 ounces', '200 grams', '60 grams', '16 ounces', '230 grams',
       '150 grams', '225 grams', '14 ounces', '240 grams', '250 grams',
       '100 grams', '50 grams', '230g', '200g', '454 grams', ' 12 ounces',
       '20 grams', ' 8 ounces', '175 grams', '285 grams', '10.5 ounces',
       '70 grams; $200.00', '120 grams', '455 grams', '113 grams',
       '400 grams', '227g', '115 grams', '24 ounces', '275 grams',
       '300 grams', '7 ounces', ' 227 grams', '5 ounces', '220 grams',
       '105 grams', '5.5 ounces', '10 capsules', '12.64 ounces',
       'six 8-ounce cans', '375 ml. flask', '150-gram tin',
       'twelve 6-ounce cans', 'six 12-ounce cans', '12-ounce bottle',
       'four 8-ounce cans', '280 ml can', '200-ml bottle',
       '160-ml bottle', '453 grams', '500 grams', '460 grams',
       '11 ounces', 'seven single-serve pouches', '20 ounces (2 types)',
       '32 ounces', '22 ounces', ' 7 ounces',

In [37]:
price = price.drop(price[(price['after_slash'] == 'six 8-ounce cans') | 
                          (price['after_slash'] == '375 ml. flask') | 
                          (price['after_slash'] == 'twelve 6-ounce cans') | 
                          (price['after_slash'] == 'six 12-ounce cans') | 
                          (price['after_slash'] == 'four 8-ounce cans') | 
                          (price['after_slash'] == '280 ml can') | 
                          (price['after_slash'] == 'six 12-ounce cans') | 
                          (price['after_slash'] == '200-ml bottle') | 
                          (price['after_slash'] == '160-ml bottle') | 
                          (price['after_slash'] == 'seven single-serve pouches') | 
                          (price['after_slash'] == 'sachet (plus one donated)') | 
                          (price['after_slash'] == 'five 5-gram single-serve packets') | 
                          (price['after_slash'] == 'eight 3.3 gram packets') | 
                          (price['after_slash'] == 'eight 5-gram tubes') | 
                          (price['after_slash'] == 'six 5-gram single-serve packets') | 
                          (price['after_slash'] == 'six 5-gram packets') | 
                          (price['after_slash'] == 'thirty 1.6-gram single-serve packets') | 
                          (price['after_slash'] == '12 capsules') | 
                          (price['after_slash'] == '24 K-Cups') | 
                          (price['after_slash'] == '16 K-cups') | 
                          (price['after_slash'] == '18 K-Cups') | 
                          (price['after_slash'] == '24 K-cups') | 
                          (price['after_slash'] == '16 K-Cups') | 
                          (price['after_slash'] == '32 K-Cups') | 
                          (price['after_slash'] == '18 capsules') | 
                          (price['after_slash'] == '18 K-Cups') | 
                          (price['after_slash'] == '24 K-cups') | 
                          (price['after_slash'] == '10 single-serve capsules') | 
                          (price['after_slash'] == '12 K-Cups') | 
                          (price['after_slash'] == '14 single-serve capsules') | 
                          (price['after_slash'] == '12 single-serve pods') | 
                          (price['after_slash'] == '12 single-serve cups') | 
                          (price['after_slash'] == '24 RealCup capsules') | 
                          (price['after_slash'] == '48 RealCup capsules') | 
                          (price['after_slash'] == '12 T Discs') | 
                          (price['after_slash'] == '22 K-Cups') | 
                          (price['after_slash'] == '108 pods')].index)


In [62]:
price = price.replace('', np.nan)  # replace empty strings with NaN
price = price.dropna(subset=['100_gr_EUR'], inplace=True)  # drop rows with NaN in '100_gr_EUR'

In [36]:
price.loc[price['100_gr_EUR'] == '']

,est_price,blind_assessment,after_slash,before_slash,before_DOL,rem_D,Euro,100_gr_EUR
0,$30.00/12 ounces,"Richly aromatic, floral-toned. Magnolia, cocoa...",12 ounces,$30.00,,30.00,28.5,
1,$160.00/12 ounces,"Crisply sweet-savory. White peach, hop flowers...",12 ounces,$160.00,,160.00,152.0,
2,$28.00/12 ounces,"Delicately aromatic, complex. Lilac, cocoa nib...",12 ounces,$28.00,,28.00,26.6,
3,$20.50/12 ounces,"High-toned, fruit-driven. Boysenberry, pear, c...",12 ounces,$20.50,,20.50,19.475,
4,$18.99/8 ounces,"Fruit-driven, crisply chocolaty. Goji berry, d...",8 ounces,$18.99,,18.99,18.0405,
...,...,...,...,...,...,...,...,...
4944,$27.00/500 grams (17.6 oz.),Evaluated as espresso. In the small cup exhila...,500 grams (17.6 oz.),$27.00,,27.00,25.65,
4945,$19.45/16 ounces,Balanced in aroma and cup. Floral tones are co...,16 ounces,$19.45,,19.45,18.4775,
4946,$10.50/250 grams (8.8 oz.),"Deep, rich and pungent in aroma and cup. Round...",250 grams (8.8 oz.),$10.50,,10.50,9.975,
4947,$13.99/16 ounces,"Gently and richly charred cedar, dark chocolat...",16 ounces,$13.99,,13.99,13.2905,


In [34]:
price['100_gr_EUR']

0        
1        
2        
3        
4        
       ..
4944     
4945     
4946     
4947     
4948     
Name: 100_gr_EUR, Length: 4870, dtype: object

we dropped 172 records

In [35]:
price['100_gr_EUR'] = price['100_gr_EUR'].astype(float)

ValueError: could not convert string to float: ''

In [33]:
price['100_gr_EUR'] = price['100_gr_EUR'].round(2)

TypeError: can't multiply sequence by non-int of type 'float'